In [14]:
import pandas as pd 
import numpy as np
import random
import os 
import re
import time
import math
import argparse
import pymysql
import json
import torch
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from torch import nn
from sklearn.utils import shuffle 
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification
from torch.utils.data import TensorDataset
from attrdict import AttrDict
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import BertConfig, BertTokenizer, BertModel
from transformers.optimization import get_cosine_schedule_with_warmup
from scipy.spatial.distance import pdist, squareform
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE 

In [2]:
os.getcwd()

'D:\\AuD\\model1-2\\src'

In [3]:
data_path = "D:/AuD/model1-2/data"
base_model_path = "D:/AuD/base-model"
save_model_path = "D:/AuD/model1-2/model/"
config_path = "D:/AuD/model1-2/config"
log_path = "D:/AuD/model1-2/log"
config_file = "bert-base.json"

In [4]:
dsm_samp = pd.read_csv(os.path.join(data_path, 'dsm_samp2_test.csv'))
dsm_samp.head(3)

,id,text,label
0,4fws4y,"anyways, the doctor says be careful because yo...",3
1,t3_23p0b1,"i also have [aka *amerge*] for migraines, but ...",3
2,26twx8,all in all i feel worthless pretty much the on...,8


In [5]:
tokenizer = BertTokenizer.from_pretrained(os.path.join(base_model_path, 'bert-small'), model_max_length=32)
config = BertConfig.from_pretrained(os.path.join(base_model_path, 'bert-small', 'bert_config.json'),\
                                    num_labels=10,\
                                    output_hidden_states=True,\
                                    output_attentions=True)
model = BertForSequenceClassification.from_pretrained(os.path.join(base_model_path, 'bert-small'), config=config)

Some weights of the model checkpoint at D:/AuD/base-model\bert-small were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n

In [6]:
with open(os.path.join(config_path, 'training_config.json')) as f:
    training_config = AttrDict(json.load(f))

training_config.pad = 'max_length'
training_config.device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

In [7]:
config.max_position_embeddings = 128

In [8]:
# model_name = os.path.join(save_model_path, 'label_10', 'bert-mini', 'bert_dsm_4.pt')
# model_name = os.path.join(save_model_path, 'label_10', 'bert-base', 'bert_dsm_1.pt')
model_name = os.path.join(save_model_path, 'label_10', 'bert-small', 'bert_dsm_1.pt')

In [9]:
model.load_state_dict(torch.load(model_name, map_location=torch.device('cpu')))
model.to(training_config.device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 512, padding_idx=0)
      (position_embeddings): Embedding(512, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-12, element

In [180]:
inputs, len(inputs[0])

(tensor([[  101,  1045,  2572,  2061, 14777,  1998,  2292,  8167, 12863,  2008,
           1045,  2514,  2066,  1045,  1005,  1049,  2012,  2188,  2035,  2154,
           1012,   102]]),
 22)

In [181]:
outputs = model(inputs)  # Run model
attention = outputs[-1]  # Retrieve attention from model outputs
tokens = tokenizer.convert_ids_to_tokens(inputs[0])  # Convert input ids to token strings

In [182]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[ 1.0085, -1.1707, -1.8969, -1.6441, -3.3119, -2.3559, -1.1055, -2.4915,
         -1.9942,  7.9496]], grad_fn=<AddmmBackward0>), hidden_states=(tensor([[[-0.1901, -0.2641, -0.0525,  ..., -0.9500, -0.2766, -0.1464],
         [ 0.1015,  0.5429,  1.8625,  ...,  1.2891, -0.2637, -0.5567],
         [ 0.1638, -0.0085, -0.4589,  ..., -0.6815, -0.2675, -0.8721],
         ...,
         [ 0.7381, -1.1474, -0.1650,  ...,  0.9194, -0.8965,  0.8303],
         [-0.0850, -0.6053, -1.1474,  ...,  0.3525, -0.0391,  0.4304],
         [ 1.5000, -0.7967,  0.8970,  ..., -0.1933, -0.7220, -0.0389]]],
       grad_fn=<NativeLayerNormBackward0>), tensor([[[ 0.0582,  0.0376, -0.2829,  ..., -0.6916, -0.3581,  0.1042],
         [ 0.1362,  1.0145,  1.7849,  ...,  0.1044, -1.0677, -0.1571],
         [-0.3139,  0.9738, -0.5034,  ..., -0.5775, -0.2390,  0.5566],
         ...,
         [ 0.5239, -0.0151, -1.3967,  ...,  1.5463, -1.6831,  0.1674],
         [-0.3254, -0

In [183]:
len(outputs.attentions), outputs.attentions[0].size()

(4, torch.Size([1, 8, 22, 22]))

In [184]:
len(outputs.hidden_states), len(outputs.hidden_states[0]), len(outputs.hidden_states[0][0]), len(outputs.hidden_states[0][0][0])

(5, 1, 22, 512)

In [185]:
len(outputs.attentions), len(outputs.attentions[-1]), len(outputs.attentions[-1][0]), len(outputs.attentions[-1][0][0]), len(outputs.attentions[-1][0][0][0])

(4, 1, 8, 22, 22)

In [186]:
torch.squeeze(outputs.attentions[-1], dim=1).size()

torch.Size([1, 8, 22, 22])

In [187]:
outputs.keys()

odict_keys(['logits', 'hidden_states', 'attentions'])

In [188]:
outputs.attentions[-1][0][0][7], sum(outputs.attentions[-1][0][0][7])

(tensor([1.3226e-01, 1.7927e-03, 2.3589e-02, 1.0825e-02, 5.1292e-03, 7.8609e-03,
         1.9151e-04, 2.4061e-03, 1.6319e-01, 3.0710e-02, 5.1223e-03, 1.5187e-02,
         4.1093e-03, 2.5076e-04, 2.5608e-03, 3.8680e-04, 5.2220e-05, 7.2891e-04,
         6.6621e-04, 1.4668e-02, 4.6547e-01, 1.1284e-01],
        grad_fn=<SelectBackward0>),
 tensor(1.0000, grad_fn=<AddBackward0>))

In [189]:
outputs.attentions[-1][0]

tensor([[[3.3376e-02, 1.2997e-05, 2.0217e-03,  ..., 1.0276e-04,
          9.5132e-01, 8.7554e-03],
         [6.9912e-02, 3.1086e-05, 1.6757e-02,  ..., 1.1323e-04,
          8.7177e-01, 3.6929e-02],
         [5.2893e-02, 2.4645e-05, 1.3960e-02,  ..., 3.4696e-04,
          8.9831e-01, 1.9358e-02],
         ...,
         [5.0593e-02, 7.8591e-06, 4.3586e-03,  ..., 4.4215e-04,
          9.1461e-01, 2.6288e-02],
         [5.3999e-02, 3.5210e-04, 8.3446e-03,  ..., 9.0901e-04,
          8.9215e-01, 1.7195e-02],
         [4.8373e-02, 5.3651e-05, 5.6925e-03,  ..., 3.3125e-04,
          9.1608e-01, 1.7486e-02]],

        [[1.9474e-01, 3.6305e-05, 5.2411e-03,  ..., 6.0044e-05,
          7.6440e-01, 2.5268e-02],
         [3.7254e-02, 3.8121e-05, 3.9001e-04,  ..., 2.2266e-04,
          9.4746e-01, 1.0236e-02],
         [1.4702e-01, 7.1843e-05, 4.5785e-02,  ..., 2.0839e-04,
          7.0856e-01, 5.1018e-02],
         ...,
         [7.5420e-02, 4.1289e-04, 3.9180e-03,  ..., 4.5965e-02,
          8.060

In [190]:
outputs.attentions[-1][0].size()

torch.Size([8, 22, 22])

In [191]:
outputs.attentions[-1][0][0][7].sort(descending=True)

torch.return_types.sort(
values=tensor([4.6547e-01, 1.6319e-01, 1.3226e-01, 1.1284e-01, 3.0710e-02, 2.3589e-02,
        1.5187e-02, 1.4668e-02, 1.0825e-02, 7.8609e-03, 5.1292e-03, 5.1223e-03,
        4.1093e-03, 2.5608e-03, 2.4061e-03, 1.7927e-03, 7.2891e-04, 6.6621e-04,
        3.8680e-04, 2.5076e-04, 1.9151e-04, 5.2220e-05],
       grad_fn=<SortBackward0>),
indices=tensor([20,  8,  0, 21,  9,  2, 11, 19,  3,  5,  4, 10, 12, 14,  7,  1, 17, 18,
        15, 13,  6, 16]))

In [315]:
att_metrics = outputs.attentions[-1][0]
att_sum = list(map(sum, att_metric))

In [211]:
sum(att_sum).sort(descending=True).indices

tensor([20,  4,  0,  7,  8, 21, 17,  9, 12,  2, 19,  5, 11,  6,  3, 18, 15, 14,
        10,  1, 16, 13])

In [203]:
sum(att_sum).sort(descending=True)

torch.return_types.sort(
values=tensor([1.1546e+02, 1.5023e+01, 1.2315e+01, 9.2965e+00, 8.4022e+00, 6.5998e+00,
        1.5970e+00, 1.0318e+00, 9.0482e-01, 8.5423e-01, 7.5215e-01, 6.3873e-01,
        6.3835e-01, 6.2507e-01, 6.0779e-01, 4.9820e-01, 2.1790e-01, 2.1514e-01,
        1.2704e-01, 8.1850e-02, 6.8447e-02, 4.1771e-02],
       grad_fn=<SortBackward0>),
indices=tensor([20,  4,  0,  7,  8, 21, 17,  9, 12,  2, 19,  5, 11,  6,  3, 18, 15, 14,
        10,  1, 16, 13]))

In [97]:
print(inputs[0][12], inputs[0][11], inputs[0][5], inputs[0][6], inputs[0][7])
tokenizer.decode(1012), tokenizer.decode(7038), tokenizer.decode(18923), tokenizer.decode(1010), tokenizer.decode(1998)

tensor(1012) tensor(7038) tensor(18923) tensor(1010) tensor(1998)


('.', 'p o u n d s', 'a p p e t i t e', ',', 'a n d')

In [207]:
inputs[0][20]

tensor(1012)

In [192]:
print(inputs[0][20], inputs[0][8], inputs[0][9], inputs[0][2], inputs[0][13])
tokenizer.decode(1012), tokenizer.decode(12863), tokenizer.decode(2008), tokenizer.decode(2572), tokenizer.decode(1045)

tensor(1012) tensor(12863) tensor(2008) tensor(2572) tensor(1045)


('.', '# # g i c', 't h a t', 'a m', 'i')

In [325]:
def get_att_toks(input_text, num_words):
    input_text = input_text.replace("'m", " am").replace('.', ' ').replace(',', ' ')
    print(input_text)
    inputs = tokenizer.encode(input_text, return_tensors='pt').to(training_config.device)
    outputs = model(inputs)  # Run model
    attention = outputs[-1]  # Retrieve attention from model outputs
    tokens = tokenizer.convert_ids_to_tokens(inputs[0])  # Convert input ids to token strings
    att_metrics = outputs.attentions[-1][0]
    # print(f'att_metric: {len(att_metrics[-1])}')
    att_sum = list(map(sum, att_metrics))
    # print(f'att_sum: {len(att_sum[0])}')
    sorted_att = sum(att_sum).sort(descending=True)
    
    cnt = 0 
    tok_idx = []
    for idx in range(len(inputs[0])):
        if inputs[0][sorted_att.indices[idx]] == 101 or inputs[0][sorted_att.indices[idx]] == 102:
            continue
        tok_idx.append(sorted_att.indices[idx])
        cnt += 1
        if cnt == num_words:
            break 
    
    tok_list = [tokenizer.decode(inputs[0][int(tok)]) for tok in tok_idx]
    return tok_list

In [344]:
input_text = "These days,I feel like I'm too worthless and I want to die."
inputs = tokenizer.encode(input_text, return_tensors='pt').to(training_config.device)
get_att_toks(input_text, 15)

These days I feel like I am too worthless and I want to die 


['w o r t h l e s s',
 'w a n t',
 't o',
 'd i e',
 't o o',
 'i',
 'a n d',
 'a m',
 'i',
 'f e e l',
 'i',
 't h e s e',
 'd a y s',
 'l i k e']